In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"
import gc
import json
import pprint
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from functools import partial
from argparse import Namespace
import matplotlib.pyplot as plt
from tabulate import tabulate

import tensorflow as tf
print("TensorFlow:", tf.__version__)
from tensorflow.keras import layers
from tensorflow.keras import models
import tensorflow_similarity as tfsim
print("TensorFlow Similarity", tfsim.__version__)

from sklearn.model_selection import train_test_split

import wandb
print("W&B", wandb.__version__)
from wandb.keras import WandbCallback

from model import SimpleSupervisedModel, ArcFaceSupervisedModel, get_feature_extractor
from config import get_train_config
from data import GetDataloader
from utils import ShowBatch, id_generator, get_stratified_k_fold
from callbacks import GetCallbacks

pp = pprint.PrettyPrinter(indent=1)

In [ ]:
args = get_train_config()

random_id = id_generator(size=8)
args.exp_id = random_id

In [ ]:
# Allow gradual growth of GPU memory
tfsim.utils.tf_cap_memory()

# Clear out any old model state.
gc.collect()
tf.keras.backend.clear_session()

In [ ]:
df = pd.read_csv('../multishot.csv')
with open('../multishot.json') as json_file:
    label2ids = json.load(json_file)
    
print(len(df), len(label2ids))
df.head()

In [ ]:
x_train, y_train = [], []

for i in tqdm(range(len(df))):
    row = df.loc[i]
    x_train.append(np.asarray(Image.open(row.img_path)))
    y_train.append(row.target)

In [ ]:
CLASSES = list(label2ids.values())
NUM_CLASSES = 32  # @param {type: "slider", min: 1, max: 6329}
CLASSES_PER_BATCH = NUM_CLASSES
EXAMPLES_PER_CLASS = 2  # @param {type:"integer"}
STEPS_PER_EPOCH = 1000  # @param {type:"integer"}

sampler = tfsim.samplers.MultiShotMemorySampler(
    x_train,
    y_train,
    classes_per_batch=CLASSES_PER_BATCH,
    examples_per_class_per_batch=EXAMPLES_PER_CLASS,
    class_list=CLASSES, # classes to be used, all of it.
    steps_per_epoch=STEPS_PER_EPOCH,
)

In [ ]:
def get_model():
    base_model = tf.keras.applications.EfficientNetB0(include_top=False, weights='imagenet')
    base_model.trainabe = True

    inputs = layers.Input((args.image_height, args.image_width, 3))
    rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1 / 255.)(inputs)
    conv_block = base_model(rescale, training=True)
    embed = layers.GlobalAveragePooling2D()(conv_block)
    embed = layers.Dropout(0.5)(embed)
    outputs = tfsim.layers.MetricEmbedding(128)(embed)
    return tfsim.models.SimilarityModel(inputs, outputs)

tf.keras.backend.clear_session()
model = get_model()
model.summary()

In [32]:
distance = "cosine"  # @param ["cosine", "L2", "L1"]{allow-input: false}
loss = tfsim.losses.MultiSimilarityLoss(distance=distance)

In [33]:
LR = 0.0001  # @param {type:"number"}
model.compile(optimizer=tf.keras.optimizers.Adam(LR), loss=loss)

Distance metric automatically set to cosine use the distance arg to override.


In [34]:
EPOCHS = 1  # @param {type:"integer"}

# run = wandb.init(project='happywhale-similarity', job_type='train')
_ = model.fit(sampler, 
              epochs=EPOCHS)
# wandb.finish()

1000/1000 [==============================] - ETA: 0s - loss: 1.2025

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'numpy.ndarray'>"}), (<class 'list'> containing values of types {"<class 'numpy.int64'>"})

In [35]:
x_index, y_index = tfsim.samplers.select_examples(x_train, y_train, CLASSES)
model.reset_index()
model.index(x_index, y_index, data=x_index)

filtering examples:   0%|          | 0/41775 [00:00<?, ?it/s]

selecting classes:   0%|          | 0/6329 [00:00<?, ?it/s]

gather examples:   0%|          | 0/41775 [00:00<?, ?it/s]

[Indexing 41775 points]
|-Computing embeddings


2022-02-20 14:40:24.557339: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2053324800 exceeds 10% of free system memory.
2022-02-20 14:40:25.752565: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2053324800 exceeds 10% of free system memory.


|-Storing data points in key value store
|-Adding embeddings to index.
|-Building index.



0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************


In [36]:
model.index_summary()

[Info]
------------------  ------
distance            cosine
key value store     memory
search algorithm    nmslib
evaluator           memory
index size          41775
calibrated          False
calibration_metric  f1
embedding_output
------------------  ------



[Performance]
-----------  -
num lookups  0
min          0
max          0
avg          0
median       0
stddev       0
-----------  -


In [29]:
# re-run to test on other examples
num_neighbors = 5

# select
x_display, y_display = tfsim.samplers.select_examples(x_train, y_train, CLASSES)

# lookup nearest neighbors in the index
nns = model.lookup(x_display, k=num_neighbors)

filtering examples:   0%|          | 0/41775 [00:00<?, ?it/s]

selecting classes:   0%|          | 0/6329 [00:00<?, ?it/s]

gather examples:   0%|          | 0/41775 [00:00<?, ?it/s]

2022-02-20 12:47:35.578621: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2053324800 exceeds 10% of free system memory.



Performing NN search



Building NN list:   0%|          | 0/41775 [00:00<?, ?it/s]

In [35]:
# # display
# for idx in np.argsort(y_display):
#     tfsim.visualization.viz_neigbors_imgs(x_display[idx], y_display[idx], nns[idx], fig_size=(16, 2), cmap="Greys")
#     if idx==10:
#         break

In [37]:
num_calibration_samples = 10000  # @param {type:"integer"}
calibration = model.calibrate(
    x_train[:num_calibration_samples],
    y_train[:num_calibration_samples],
    extra_metrics=["precision", "recall", "binary_accuracy"],
    verbose=1,
)

ValueError: in user code:

    /opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1586 predict_function  *
        return step_function(self, iterator)
    /opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1576 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1569 run_step  **
        outputs = model.predict_step(data)
    /opt/conda/lib/python3.7/si